# Resolución en Frecuencia y Ventaneo

## Discrete Time Fourier Transform (DTFT) y Ventaneo

Al analizar señales en la práctica, no podemos procesar una secuencia infinita de muestras $x(n)$, $-\infty < n < \infty$. La Transformada Discreta de Fourier en Tiempo (DTFT), definida como:

$$ \hat{X}(\omega) = \sum_{n=-\infty}^{\infty} x(n)e^{-j\omega n} $$

es teóricamente útil pero no computable directamente.

En la práctica, limitamos nuestro análisis a una ventana finita de $L$ muestras, por ejemplo, $0 \le n \le L-1$. Esto se llama **ventaneo (windowing)**. La DTFT que *sí* podemos calcular se basa en esta señal ventaneada $x_L(n)$:

$$ \hat{X}_L(\omega) = \sum_{n=0}^{L-1} x(n)e^{-j\omega n} = \sum_{n=-\infty}^{\infty} x_L(n)e^{-j\omega n} $$

donde $x_L(n) = x(n) \cdot w(n)$, siendo $w(n)$ la función ventana (por ejemplo, rectangular o Hamming).

## Efectos del Ventaneo

Aplicar una ventana (multiplicar $x(n)$ por $w(n)$ en el tiempo) tiene dos efectos principales en el espectro de frecuencia $\hat{X}_L(\omega)$:

1.  **Resolución en Frecuencia Limitada:** La duración finita $T_L = LT = L/f_s$ de la ventana limita nuestra capacidad para distinguir (resolver) dos componentes de frecuencia muy cercanas. La mínima separación en frecuencia resoluble es aproximadamente:
    $$ \Delta f \approx \frac{1}{T_L} = \frac{f_s}{L} \quad [\text{Hz}] $$
    o en frecuencia digital:
    $$ \Delta \omega_w \approx \frac{2\pi}{L} \quad [\text{rad/muestra}] $$
    Para resolver dos frecuencias $\omega_1$ y $\omega_2$, necesitamos que $|\omega_1 - \omega_2| \ge \Delta \omega_w$. Una ventana más larga (mayor $L$) mejora la resolución (menor $\Delta f$ o $\Delta \omega_w$).

2.  **Fuga Espectral (Frequency Leakage):** Multiplicar por $w(n)$ en el tiempo equivale a convolucionar con la transformada de Fourier de la ventana, $\hat{W}(\omega)$, en frecuencia:
    $$ \hat{X}_L(\omega) = \frac{1}{2\pi} (\hat{X} * \hat{W})(\omega) $$
    La transformada $\hat{W}(\omega)$ de una ventana (como la rectangular) tiene un lóbulo principal ancho y varios lóbulos laterales más pequeños. La convolución con esta forma "esparce" o "derrama" la energía de una frecuencia pura (que idealmente sería un delta $\delta(\omega - \omega_0)$) sobre un rango de frecuencias definido por $\hat{W}(\omega)$. Los lóbulos laterales son responsables de la **fuga espectral**, donde la energía de un componente fuerte puede enmascarar componentes más débiles en frecuencias cercanas.

In [1]:
# --------------- Imports ---------------
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import IntSlider, Dropdown, VBox, interactive_output, Layout
from scipy.signal import get_window
import math

# --- Constantes y Funciones ---
FS = 1000 # Frecuencia de muestreo simulada (Hz) para referencia
F0 = 50   # Frecuencia de la sinusoide (Hz)
W0 = 2 * math.pi * F0 / FS # Frecuencia digital (rad/muestra) = 0.1*pi

def calculate_spectrum(signal, n_fft=2048):
    """Calcula la FFT y la prepara para graficar (magnitud en dB)"""
    # Padding para mejor visualización del espectro
    padded_signal = np.pad(signal, (0, n_fft - len(signal)), 'constant')
    fft_result = np.fft.fft(padded_signal)
    fft_freq = np.fft.fftfreq(n_fft, d=1/FS) # Frecuencia en Hz

    # Tomamos la mitad positiva del espectro
    half_point = n_fft // 2
    fft_freq_half = fft_freq[:half_point]
    fft_mag_half = np.abs(fft_result[:half_point])

    # Convertir a dB, evitando log(0)
    fft_db = 20 * np.log10(np.maximum(fft_mag_half, 1e-9)) # Usamos 1e-9 como piso
    fft_db -= np.max(fft_db) # Normalizar a 0 dB el pico máximo
    return fft_freq_half, fft_db

# --- Widgets de Control ---
style = {'description_width': 'initial'}
layout_slider = Layout(width='80%')

L_slider_ex1 = IntSlider(min=50, max=400, step=10, value=100,
                         description='Longitud Ventana (L):', style=style, layout=layout_slider)
window_dropdown_ex1 = Dropdown(options=['rectangular', 'hamming'], value='rectangular',
                               description='Tipo Ventana:', style=style, layout=layout_slider)

# --- Contenedor para el gráfico ---
plot_output_ex1 = widgets.Output()

# --- Función de Actualización y Creación de Gráfico ---
def update_plot_ex1(L, window_type):
    # 1. Generar señal y ventana
    n = np.arange(L)
    signal = np.cos(W0 * n)
    window = get_window(window_type, L)
    windowed_signal = signal * window

    # 2. Calcular espectro
    freq_axis, spectrum_db = calculate_spectrum(windowed_signal)
    resolution_hz = FS / L
    resolution_rad = 2 * math.pi / L

    # 3. Crear figura
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=('Señal Ventaneada en Tiempo',
                                        f'Espectro Magnitud |X_L(f)| (Resolución ~ {resolution_hz:.2f} Hz ó {resolution_rad:.4f} rad/muestra)'))

    # Gráfico señal en tiempo
    fig.add_trace(go.Scatter(x=n, y=windowed_signal, mode='lines', name='x_L(n)'), row=1, col=1)
    fig.update_xaxes(title_text="n (muestras)", row=1, col=1)
    fig.update_yaxes(title_text="Amplitud", range=[-1.1, 1.1], row=1, col=1)

    # Gráfico espectro en frecuencia (eje X en Hz)
    fig.add_trace(go.Scatter(x=freq_axis, y=spectrum_db, mode='lines', name='|X_L(f)| dB'), row=2, col=1)
    fig.add_vline(x=F0, line_width=1, line_dash="dash", line_color="red", annotation_text="f0=50Hz", row=2, col=1)
    fig.update_xaxes(title_text="f (Hz)", range=[0, F0 * 4], row=2, col=1) # Mostrar hasta 4*F0
    fig.update_yaxes(title_text="Magnitud (dB)", range=[-100, 5], row=2, col=1) # Rango típico en dB

    fig.update_layout(height=600, title_text=f"Ventana: {window_type}, Longitud L={L}", showlegend=False)

    # Limpiar y mostrar
    with plot_output_ex1:
        plot_output_ex1.clear_output(wait=True)
        fig.show()

# --- Conectar Widgets y Mostrar ---
out_ex1 = interactive_output(update_plot_ex1, {'L': L_slider_ex1, 'window_type': window_dropdown_ex1})

print("Ejemplo de Clase 11: Efecto de L y tipo de ventana en el espectro de un coseno.")
display(VBox([window_dropdown_ex1, L_slider_ex1, plot_output_ex1]))

# Mostrar gráfico inicial
update_plot_ex1(L_slider_ex1.value, window_dropdown_ex1.value)

print("\n--- Fin del Ejemplo 1 ---")

Ejemplo de Clase 11: Efecto de L y tipo de ventana en el espectro de un coseno.



--- Fin del Ejemplo 1 ---


## Ejemplo 2: Resolución de dos Sinusoides Cercanas en Ruido

Este ejemplo ilustra cómo la longitud de la ventana (L), el tipo de ventana, la separación en frecuencia ($\Delta f$), la amplitud relativa (A2) y el ruido afectan nuestra capacidad para distinguir dos sinusoides cercanas.

**Gráficos Interactivos:**
1.  **Señal de Entrada Ventaneada $x_L(n)$:** Muestra la porción de la señal que realmente se analiza ($x(n) = \cos(\omega_1 n) + A_2 \cos(\omega_2 n) + v(n)$ multiplicada por la ventana $w(n)$).
2.  **Función Ventana $w(n)$:** Muestra la forma de la ventana seleccionada.
3.  **Espectro $|X_L(f)|$:** Muestra el resultado en frecuencia (calculado mediante FFT).

In [2]:
# --------------- Imports ---------------
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Dropdown, VBox, interactive_output, Layout
from scipy.signal import get_window
import math

# --- Constantes y Funciones ---
FS = 1000 # Hz (Frecuencia de muestreo simulada)
np.random.seed(420) # Para reproducibilidad

def calculate_spectrum_ex2(signal, n_fft=2048):
    """Calcula la FFT y la prepara para graficar (magnitud en dB)"""
    if len(signal) < 1: 
        return np.array([]), np.array([])
    current_n_fft = max(n_fft, 2**(math.ceil(math.log2(len(signal))))) #
    if len(signal) < current_n_fft:
        padded_signal = np.pad(signal, (0, current_n_fft - len(signal)), 'constant')
    else:
        padded_signal = signal[:current_n_fft]

    fft_result = np.fft.fft(padded_signal)
    fft_freq = np.fft.fftfreq(current_n_fft, d=1/FS) # Frecuencia en Hz

    half_point = current_n_fft // 2
    fft_freq_half = fft_freq[:half_point]
    fft_mag_half = np.abs(fft_result[:half_point])

    # Convertir a dB, evitando log(0) y errores
    valid_indices = fft_mag_half > 1e-10
    fft_db = np.full_like(fft_mag_half, -150.0) # Piso de ruido muy bajo en dB
    fft_db[valid_indices] = 20 * np.log10(fft_mag_half[valid_indices])

    max_db = np.max(fft_db[np.isfinite(fft_db)]) # Maximo finito
    if np.isfinite(max_db):
         fft_db -= max_db # Normalizar a 0 dB
    else:
         fft_db[:] = -150.0 # Si todo es muy pequeño, mantener el piso

    return fft_freq_half, fft_db

# --- Widgets de Control ---
style = {'description_width': 'initial'}
layout_widget_ex2 = Layout(width='90%')

L_slider_ex2 = IntSlider(min=50, max=1000, step=10, value=200,
                         description='Longitud Ventana (L):', style=style, layout=layout_widget_ex2)
window_dropdown_ex2 = Dropdown(options=['rectangular', 'hamming', 'hann', 'blackman'], value='rectangular',
                               description='Tipo Ventana:', style=style, layout=layout_widget_ex2)
delta_f_slider = FloatSlider(min=1.0, max=50.0, step=1.0, value=10.0,
                             description='Separación Frec. Δf (Hz):', style=style, layout=layout_widget_ex2, readout_format='.1f')
A2_slider = FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5,
                        description='Amplitud A2 (relativa a A1=1):', style=style, layout=layout_widget_ex2)
noise_slider = FloatSlider(min=0.0, max=0.5, step=0.01, value=0.05,
                           description='Nivel Ruido σ_v:', style=style, layout=layout_widget_ex2)

# --- Contenedor para el gráfico ---
plot_output_ex2 = widgets.Output()

# --- Función de Actualización y Creación de Gráfico ---
def update_plot_ex2_final(L, window_type, delta_f, A2, noise_std):
    # 1. Generar señal base
    F1 = 100 # Hz
    F2 = F1 + delta_f # Hz
    W1 = 2 * math.pi * F1 / FS
    W2 = 2 * math.pi * F2 / FS
    n = np.arange(L)
    s1 = np.cos(W1 * n)
    s2 = A2 * np.cos(W2 * n)
    noise = np.random.normal(0, noise_std, L) if noise_std > 0 else np.zeros(L)
    signal_original = s1 + s2 + noise # Guardar la señal original

    # 2. Aplicar ventana
    window = get_window(window_type, L)
    windowed_signal = signal_original * window # Esta es x_L(n)

    # 3. Calcular espectro
    freq_axis, spectrum_db = calculate_spectrum_ex2(windowed_signal)

    # Calcular resolución aproximada
    c_factor = {'rectangular': 1.0, 'hamming': 2.0, 'hann': 2.0, 'blackman': 3.0}.get(window_type, 1.0)
    resolution_hz = c_factor * FS / L if L > 0 else float('inf')

    # 4. Crear figura con 4 subplots
    fig = make_subplots(rows=4, cols=1,
                        shared_xaxes=False,
                        vertical_spacing=0.08, # Ajustar espacio
                        subplot_titles=(f'1. Señal Original x(n) (L={L})',
                                        f'2. Ventana w(n) ({window_type})',
                                        f'3. Señal Ventaneada x_L(n) = x(n)w(n)',
                                        f'4. Espectro Magnitud |X_L(f)| (Resolución ~ {resolution_hz:.2f} Hz)'))

    # Gráfico 1: Señal original en tiempo
    fig.add_trace(go.Scatter(x=n, y=signal_original, mode='lines', name='x(n)', line=dict(color='grey')), row=1, col=1)
    fig.update_xaxes(title_text="n (muestras)", row=1, col=1)
    max_abs_orig = np.max(np.abs(signal_original)) if L > 0 else 1
    fig.update_yaxes(title_text="Amplitud", range=[-max_abs_orig*1.1, max_abs_orig*1.1], row=1, col=1)

    # Gráfico 2: Función ventana
    fig.add_trace(go.Scatter(x=n, y=window, mode='lines', name='w(n)', line=dict(color='purple')), row=2, col=1)
    fig.update_xaxes(title_text="n (muestras)", row=2, col=1)
    fig.update_yaxes(title_text="w(n)", range=[-0.1, 1.1], row=2, col=1)

    # Gráfico 3: Señal ventaneada en tiempo
    fig.add_trace(go.Scatter(x=n, y=windowed_signal, mode='lines', name='x_L(n)', line=dict(color='blue')), row=3, col=1)
    fig.update_xaxes(title_text="n (muestras)", row=3, col=1)
    max_abs_win = np.max(np.abs(windowed_signal)) if L > 0 else 1
    fig.update_yaxes(title_text="Amplitud", range=[-max_abs_win*1.1, max_abs_win*1.1], row=3, col=1)

    # Gráfico 4: Espectro en frecuencia (eje X en Hz)
    fig.add_trace(go.Scatter(x=freq_axis, y=spectrum_db, mode='lines', name='|X_L(f)| dB'), row=4, col=1)
    fig.add_vline(x=F1, line_width=1, line_dash="dash", line_color="red", annotation_text=f"f1={F1}Hz", row=4, col=1)
    fig.add_vline(x=F2, line_width=1, line_dash="dash", line_color="magenta", annotation_text=f"f2={F2:.1f}Hz", row=4, col=1)
    fig.update_xaxes(title_text="f (Hz)", range=[max(0, F1 - 5*delta_f_slider.max), F2 + 5*delta_f_slider.max], row=4, col=1)
    fig.update_yaxes(title_text="Magnitud (dB)", range=[-80, 5], row=4, col=1) # Rango dB ajustado

    fig.update_layout(height=950, # Aumentar altura para 4 plots
                      title_text=f"Resolución de Sinusoides (L={L}, Ventana: {window_type}, A2={A2:.2f}, σ_v={noise_std:.2f})",
                      showlegend=False)

    # Limpiar y mostrar
    with plot_output_ex2:
        plot_output_ex2.clear_output(wait=True)
        fig.show()

# --- Conectar Widgets y Mostrar ---
out_ex2_final = interactive_output(update_plot_ex2_final, {
    'L': L_slider_ex2,
    'window_type': window_dropdown_ex2,
    'delta_f': delta_f_slider,
    'A2': A2_slider,
    'noise_std': noise_slider
})

print("Ejemplo 2 (Versión Final): Resolución de dos sinusoides cercanas.")
controls_box_ex2 = VBox([window_dropdown_ex2, L_slider_ex2, delta_f_slider, A2_slider, noise_slider])
display(VBox([controls_box_ex2, plot_output_ex2]))

# Mostrar gráfico inicial
update_plot_ex2_final(L_slider_ex2.value, window_dropdown_ex2.value, delta_f_slider.value, A2_slider.value, noise_slider.value)

print("\n--- Fin del Ejemplo 2 ---")

Ejemplo 2 (Versión Final): Resolución de dos sinusoides cercanas.



--- Fin del Ejemplo 2 ---
